自己生成ペアのテキストを埋め込み化する

personal_path.jsonとneutral_pairs.jsonを読み込み各レコードのpP,or pNをベクトル化する

In [3]:
import json
from openai import OpenAI
import numpy as np

client = OpenAI()

with open("/home/nakata/master_thesis/rango/processed/LaMP-2/personal_pairs.json", "r", encoding="utf-8") as f:
    pp = json.load(f)

texts = [r["pP"] for r in pp]
resp = client.embeddings.create(model="text-embedding-ada-002", input = texts)
personal_embs = np.vstack([np.array(d.embedding) for d in resp.data])

In [7]:
with open("/home/nakata/master_thesis/rango/processed/LaMP-2/neutral_pairs.json", "r", encoding="utf-8") as f:
    pn = json.load(f)
texts = [r["pN"] for r in pn]
resp = client.embeddings.create(model="text-embedding-ada-002", input = texts)
neutral_embs = np.vstack([np.array(d.embedding) for d in resp.data])

In [9]:
np.save("neutral_embeddings.npy", neutral_embs)
np.save("personal_embeddings.npy", personal_embs)

neutral_pairsとpersonal_pairs embneddingが終わったので、これで何ができるようになったか？
→最終的なタグ付けがわかった・あるユーザの映画感想文を元にした。その感想文から読み取れるパーソナルな好みの情報と、一般的とみられるneutralな情報の文章OO_insights.jsonを元にしてtagを生成したOO_pairs.jsonをベクトル化した

In [11]:
import numpy as np

E_p = np.load("/home/nakata/master_thesis/rango/processed/LaMP-2/personal_embeddings.npy")
E_n = np.load("/home/nakata/master_thesis/rango/processed/LaMP-2/neutral_embeddings.npy")

E_p_centered = E_p - E_p.mean(axis=0, keepdims=True)
E_n_centered = E_n - E_n.mean(axis=0, keepdims=True)
# SVDを実行して、第一右特異ベクトルをtheta_p, theta_nとする
# np.linalg.svdはU,Σ, Vhを返す。　Vhの0行目が第一右特異ベクトル
_, _, Vh_p = np.linalg.svd(E_p_centered, full_matrices=False)
theta_P = Vh_p[0]

_, _, Vh_n = np.linalg.svd(E_p_centered,full_matrices=False)
theta_N = Vh_n[0]


In [12]:
# 正規化(任意)
theta_P /= np.linalg.norm(theta_P)
theta_N /= np.linalg.norm(theta_N)

In [13]:
print("personal direction Theta_P:", theta_P)
print("neutral direction Theta_N:", theta_N)

personal direction Theta_P: [ 0.05109705  0.00028158 -0.03811706 ...  0.01031596  0.00448704
 -0.0027529 ]
neutral direction Theta_N: [ 0.05109705  0.00028158 -0.03811706 ...  0.01031596  0.00448704
 -0.0027529 ]


In [14]:
theta_P.size

1536

In [15]:
np.save("theta_p.npy", theta_P)
np.save("theta_n.npy", theta_N)

In [16]:
import json

with open("theta_p.json", "w", encoding="utf-8") as f:
    json.dump(theta_P.tolist(), f, ensure_ascii=False, indent=2)

with open("theta_n.json", "w", encoding="utf-8") as f:
    json.dump(theta_N.tolist(), f, ensure_ascii=False, indent=2)


In [7]:
import json
from openai import OpenAI
import numpy as np

client = OpenAI()
new_review = "この映画は深い心理描写とスリルのある展開が印象的だった"
resp = client.embeddings.create(
    model = "text-embedding-ada-002",
    input = "new_review"
)

new_embedding = np.array(resp.data[0].embedding)
print("新しいレビューの embedding shape:", new_embedding.shape)
# → (1536,) (テキスト埋め込みの次元数)



新しいレビューの embedding shape: (1536,)


In [2]:
!pip show openai

Name: openai
Version: 1.61.0
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License-Expression: Apache-2.0
Location: /home/nakata/anaconda3/lib/python3.11/site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: alpaca_eval, langchain-openai


In [3]:
!pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.0
    Uninstalling openai-1.61.0:
      Successfully uninstalled openai-1.61.0

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0
    Uninstalling pip-25.0:
      Successfully uninstalled pip-25.0
Note: you may need to restart the kernel to use updated packages.
